# Handle and check the 'data' of the all the scans we did
Wrestle with the data, check parameters and generate some helping files

In [1]:
import platform
import os
import glob
import pandas
import imageio
import numpy
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar
import seaborn
import dask
import dask_image.imread
from dask.distributed import Client, LocalCluster
from numcodecs import Blosc
import skimage
from tqdm import notebook

In [2]:
# Set dask temporary folder
# Do this before creating a client: https://stackoverflow.com/a/62804525/323100
import tempfile
if 'Linux' in platform.system():
    tmp = os.path.join(os.sep, 'media', 'habi', 'Fast_SSD')
elif 'Darwin' in platform.system():
    tmp = tempfile.gettempdir()
else:
    if 'anaklin' in platform.node():
        tmp = os.path.join('F:\\')
    else:
        tmp = os.path.join('D:\\')
dask.config.set({'temporary_directory': os.path.join(tmp, 'tmp')})
print('Dask temporarry files go to %s' % dask.config.get('temporary_directory'))

Dask temporarry files go to /media/habi/Fast_SSD/tmp


In [3]:
# Start cluster and client now, after setting tempdir
cluster = LocalCluster(n_workers=8)
client = Client(cluster)

/home/habi/miniconda3/lib/python3.9/site-packages/distributed/node.py:181: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 38679 instead
  warnings.warn(


In [4]:
print('You can seee what DASK is doing at "http://localhost:%s/status"' % client.scheduler_info()['services']['dashboard'])

You can seee what DASK is doing at "http://localhost:38679/status"


In [5]:
# # Ignore warnings in the notebook
# import warnings
# warnings.filterwarnings("ignore")

In [6]:
# Set up figure defaults
plt.rc('image', cmap='gray', interpolation='nearest')  # Display all images in b&w and with 'nearest' interpolation
plt.rcParams['figure.figsize'] = (16, 9)  # Size up figures a bit
plt.rcParams['figure.dpi'] = 200

In [7]:
# Setup scale bar defaults
plt.rcParams['scalebar.location'] = 'lower right'
plt.rcParams['scalebar.frameon'] = False
plt.rcParams['scalebar.color'] = 'white'

In [8]:
# Display all plots identically
lines = 3
# And then do something like
# plt.subplot(lines, numpy.ceil(len(Data) / float(lines)), c + 1)

In [9]:
# Different locations if running either on Linux or Windows
Archive = False # Load the data directly from the iee-research_storage drive
# to speed things up significantly
if Archive:
    if 'Linux' in platform.system():
        BasePath = os.path.join(os.sep, 'home', 'habi', 'research-storage-uct', 'Archiv_Tape')
    elif 'Windows' in platform.system():
        BasePath = os.path.join('R:\\Archiv_Tape')
else:
    BasePath = os.path.join(os.getcwd(), 'Data')
Root = os.path.join(BasePath, 'Liver-Semela')
print('We are loading all the data from %s' % Root)

We are loading all the data from /home/habi/P/Documents/Semela-Liver/Data/Liver-Semela


In [10]:
def get_pixelsize(logfile):
    """Get the pixel size from the scan log file"""
    pixelsize=None    
    with open(logfile, 'r') as f:
        for line in f:
            if 'Image Pixel' in line and 'Scaled' not in line:
                pixelsize = float(line.split('=')[1])
    return(pixelsize)

In [11]:
def get_operator(logfile):
    """Get the operator who scanned the samples"""
    operator = None
    with open(logfile, 'r') as f:
        for line in f:
            if 'User Name' in line:
                operator = line.split('=')[1].strip()
    return(operator)

In [12]:
def get_experiment(i):
    '''Categorize  into 'Notch' or 'Control' '''
    if 'notch' in i:
        return 'Notch'
    if 'ctrl' in i:
        return 'Control'

In [13]:
def get_vein(i):
    if 'portal' in i:
        return 'Portal'
    elif 'cava' in i:
        return 'Cava'
    else:
        return None

In [14]:
def get_git_hash():
    '''
    Get the current git hash from the repository.
    Based on http://stackoverflow.com/a/949391/323100 and
    http://stackoverflow.com/a/18283905/323100
    '''
    from subprocess import Popen, PIPE
    import os
    gitprocess = Popen(['git',
                        '--git-dir',
                        os.path.join(os.getcwd(), '.git'),
                        'rev-parse',
                        '--short',
                        '--verify',
                        'HEAD'],
                       stdout=PIPE)
    (output, _) = gitprocess.communicate()
    return output.strip().decode("utf-8")

In [15]:
# # Make directory for output
# OutPutDir = os.path.join(os.getcwd(), 'Output', get_git_hash())
# print('We are saving all the output to %s' % OutPutDir)
# os.makedirs(OutPutDir, exist_ok=True)

Mario Novkovic told us that 
> We have used the ds17 livers in the paper, specifically ctrl4 and notch1_2 in the first batch (training dataset), while the second batch consisted of 3 datasets from each mouse type: ctrl1, ctrl2, ctrl5 and notch1_1, notch1_3, notch1_4.

So let's only use *those* folders for the remainder of the notebook.
We copied all the relevant data from the archive to the `Data`-subfolder here with
````bash
rsync --verbose --recursive --times --update --omit-dir-times --include="*/" --include="*.?og" --include="*.c?v" --include="*.?oi" --include="*.?at" --include="*_spr*.bmp" --include="*.txt" --include="*.md" --include="*.mp" --include="*.sb" --include="*.info" --include="*.?nc" --include="*.bkp" --exclude="*" ~/research-storage-uct/Archiv_Tape/Liver-Semela/ /home/habi/P/Documents/Semela-Liver/Data/

````
(which is our standard `rsync` blurb for putting stuff *to* the archive, but without the `*.tif` files, so we get back all the relevant things :).

We then delete all non-`ds17*`-folders with
````bash
find . -path './ds*' -prune -o -name '*' -delete -depth
````
(which does issue a warning because of `-depth`, but leaves us with only the `ds17_*`-folders :) )

In [16]:
# These are the folders that were used according to Mario.
whichones = ['ctrl4', 'notch1_2', 'ctrl1', 'ctrl2', 'ctrl5', 'notch1_1', 'notch1_3', 'notch1_4']

Now that we have *all* the necessary data (and some more), let's get to work!

In [17]:
# Make us a dataframe for saving all that we need
Data = pandas.DataFrame()

In [18]:
# Get *all* log files
Data['LogFile'] = [f for f in sorted(glob.glob(os.path.join(Root, '**', '*.log'), recursive=True))]

In [19]:
# Get all folders and generate sample, scan and experiment name
Data['Folder'] = [os.path.dirname(f) for f in Data['LogFile']]
Data['SampleName'] = [f[len(Root):].split(os.path.sep)[1].replace('ds17_','') for f in Data['Folder']]
Data['Sample'] = [sn.replace('_rescan','').replace('_portal','').replace('_cava','') for sn in Data['SampleName']]
Data['Scan'] = [f[len(Root):].split(os.path.sep)[2] for f in Data['Folder']]
Data['Subfolder'] = [f[len(Root):].split(os.path.sep)[3] for f in Data['Folder']]
Data['Experiment'] = [get_experiment(s) for s in Data['Sample']]
Data['Vein'] = [get_vein(s) for s in Data['SampleName']]

In [20]:
# Check what we did there...
print(Data.iloc[33])
print(80*'-')
for i in Data.iloc[33]:
    print(i)

LogFile       /home/habi/P/Documents/Semela-Liver/Data/Liver...
Folder        /home/habi/P/Documents/Semela-Liver/Data/Liver...
SampleName                                         ctrl3_portal
Sample                                                    ctrl3
Scan                                                   overview
Subfolder                                                  proj
Experiment                                              Control
Vein                                                     Portal
Name: 33, dtype: object
--------------------------------------------------------------------------------
/home/habi/P/Documents/Semela-Liver/Data/Liver-Semela/ds17_ctrl3_portal/overview/proj/ds17_ctrl3_portal.log
/home/habi/P/Documents/Semela-Liver/Data/Liver-Semela/ds17_ctrl3_portal/overview/proj
ctrl3_portal
ctrl3
overview
proj
Control
Portal


In [21]:
# Read the voxelsize from each logfile
Data['Voxelsize'] = [get_pixelsize(log) for log in Data['LogFile']]

In [22]:
# One log file is empty, drop it
# We cannot read a voxel size out of it...
Data.drop(Data[Data['Voxelsize'].isna()].index, inplace=True)
# We asked Mario et al. if they have a good copy of it.

In [23]:
Data['Operator'] = [get_operator(log) for log in Data['LogFile']]
print('The scans were performed by %s' % Data.Operator.unique())

The scans were performed by ['haberthu']


In [24]:
Data[['Folder', 'SampleName', 'Sample', 'Vein', 'Scan', 'Subfolder']]

,Folder,SampleName,Sample,Vein,Scan,Subfolder
0,/home/habi/P/Documents/Semela-Liver/Data/Liver...,ctrl1_portal,ctrl1,Portal,highresolution,proj
1,/home/habi/P/Documents/Semela-Liver/Data/Liver...,ctrl1_portal,ctrl1,Portal,highresolution,proj
2,/home/habi/P/Documents/Semela-Liver/Data/Liver...,ctrl1_portal,ctrl1,Portal,highresolution,proj
3,/home/habi/P/Documents/Semela-Liver/Data/Liver...,ctrl1_portal,ctrl1,Portal,highresolution,rec
4,/home/habi/P/Documents/Semela-Liver/Data/Liver...,ctrl1_portal,ctrl1,Portal,overview,proj
...,...,...,...,...,...,...
131,/home/habi/P/Documents/Semela-Liver/Data/Liver...,notch1_4,notch1_4,None,rll_repeat_3um,rec
132,/home/habi/P/Documents/Semela-Liver/Data/Liver...,notch1_4_rescan,notch1_4,None,highresolution,proj
133,/home/habi/P/Documents/Semela-Liver/Data/Liver...,notch1_4_rescan,notch1_4,None,highresolution,proj
134,/home/habi/P/Documents/Semela-Liver/Data/Liver...,notch1_4_rescan,notch1_4,None,highresolution,proj


In [25]:
# What samples do we have?
for i in Data.Sample.unique():
    print(i)

ctrl1
ctrl2
ctrl3
ctrl4
ctrl5
ctrl6
notch1_1
notch1_2
notch1_3
notch1_4


In [26]:
# What Mario et al looked at
sorted(whichones)

['ctrl1',
 'ctrl2',
 'ctrl4',
 'ctrl5',
 'notch1_1',
 'notch1_2',
 'notch1_3',
 'notch1_4']

In [27]:
# Which ones are 'surplus'?
set(Data.Sample.unique())-set(whichones)

{'ctrl3', 'ctrl6'}

In [28]:
# Collate what Mario analyzed with what we scanned
for wanted in sorted(whichones):
    print(15 * '-', 'For sample %s we have the data below' % wanted, 15 * '-', )
    print(Data[Data['Sample'].str.contains(wanted)][['Sample', 'Vein', 'SampleName', 'Scan', 'Voxelsize']])
    print(80*'-')

--------------- For sample ctrl1 we have the data below ---------------
   Sample    Vein           SampleName            Scan  Voxelsize
0   ctrl1  Portal         ctrl1_portal  highresolution   5.000040
1   ctrl1  Portal         ctrl1_portal  highresolution   5.000040
2   ctrl1  Portal         ctrl1_portal  highresolution   5.000040
3   ctrl1  Portal         ctrl1_portal  highresolution   5.000040
4   ctrl1  Portal         ctrl1_portal        overview  21.754494
5   ctrl1  Portal         ctrl1_portal        overview  21.101977
6   ctrl1  Portal         ctrl1_portal        overview  21.754494
8   ctrl1  Portal         ctrl1_portal             rll   3.000006
9   ctrl1  Portal         ctrl1_portal             rll   3.000006
10  ctrl1  Portal         ctrl1_portal             rll   3.000006
11  ctrl1  Portal         ctrl1_portal             rll   3.000006
12  ctrl1  Portal         ctrl1_portal         rll_5um   5.000024
13  ctrl1  Portal         ctrl1_portal         rll_5um   5.000024
14  

So, from just looking throught the data based on the names that we have from Mario, it seems to me that they *only* looked at the 5 um scans.
Let's `.drop()` all the other scans.

In [29]:
# Get rid of all non-5um scans
Data.drop(Data[Data['Voxelsize']>5.5].index, inplace=True)
Data.drop(Data[Data['Voxelsize']<4.5].index, inplace=True)

So it seems that Mario et al. simply used the `highresolution` scans (either the scan or the `_rescan`).
Now, let's get to work with extracting the data we need...

In [30]:
import re
def scanner(logfile, verbose=False):
    hardwareversion = []
    with open(logfile, 'r') as f:
        for line in f:
            if 'Scanner' in line:
                if verbose:
                    print(line)
                # Sometimes it's SkyScan, sometimes Skyscan, so we have to regex it :)
                machine = re.split('Sky.can', line)[1].strip()
            if 'Hardware' in line:
                if verbose:
                    print(line)
                hardwareversion = line.split('=')[1].strip()
    if hardwareversion:
        return('SkyScan %s (Version %s)' % (machine, hardwareversion))
    else:
        return('SkyScan ' + machine)    

In [31]:
def controlsoftware(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Software Ver' in line:
                if verbose:
                    print(line)
                version = line.split('=')[1].strip()
    return(version)

In [32]:
# Get parameters which we'll need for the paragraph
Data['Scanner'] = [scanner(log) for log in Data['LogFile']]
Data['ControlSoftware'] = [controlsoftware(log) for log in Data['LogFile']]

In [33]:
def source(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Source Ty' in line:
                if verbose:
                    print(line)
                source = line.split('=')[1].strip()
                if 'HAMAMA' in source:
                    # We split the string at '_L' to separate HAMAMATSU_L118...
                    # Afterwards we properly capitalize HAMAMATSU and
                    # join the strings back with ' L' to get the beginning of the reference back
                    source = ' L'.join([s.capitalize() for s in source.split('_L')])
    return(source)

In [34]:
def camera(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Camera T' in line or 'Camera=' in line:
                if verbose:
                    print(line)
                cam = line.split('=')[1].strip().strip(' camera')
    return(cam)

In [35]:
Data['Source'] = [source(log) for log in Data['LogFile']]
Data['Camera'] = [camera(log) for log in Data['LogFile']]

In [36]:
def voltage(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Voltage' in line:
                if verbose:
                    print(line)
                V = float(line.split('=')[1])
    return(V)

In [37]:
def current(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Source Current' in line:
                if verbose:
                    print(line)
                A = float(line.split('=')[1])
    return(A)

In [38]:
def whichfilter(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Filter=' in line:
                if verbose:
                    print(line)
                fltr = line.split('=')[1].strip().replace('  ', ' ')
                if fltr=='No Filter':
                    fltr=False
    return(fltr)

In [39]:
Data['Voltage'] = [voltage(log) for log in Data['LogFile']]
Data['Current'] = [current(log) for log in Data['LogFile']]
Data['Filter'] = [whichfilter(log) for log in Data['LogFile']]

In [40]:
def camerasize(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Columns' in line:
                if verbose:
                    print(line)
                columns = int(line.split('=')[1])
            if 'Rows' in line:
                if verbose:
                    print(line)
                rows = int(line.split('=')[1])
    return(columns, rows)

In [41]:
def numproj(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'f Files' in line:
                if verbose:
                    print(line)
                numproj = int(line.split('=')[1])
    return(numproj)

In [42]:
def rotationstep(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Rotation Step' in line:
                if verbose:
                    print(line)
                rotstep = float(line.split('=')[1])
    return(rotstep)

In [43]:
Data['CamSize'] = [camerasize(log) for log in Data['LogFile']]
Data['NumProj'] = [numproj(log) for log in Data['LogFile']]
Data['RotationStep'] = [rotationstep(log) for log in Data['LogFile']]

In [44]:
def stacks(logfile, verbose=False):
    with open(logfile, 'r') as f:
        # If only one stack, then there's nothing in the log file
        numstacks = 0
        for line in f:
            if 'of conn' in line:
                if verbose:
                    print(line)
                # The 'Sub-scan scan length' is listed in the log file
                # We simply select the last one, and add 1, since Bruker also starts to count at zero
                numstacks = int(line.split('=')[1])
    return(numstacks)

In [45]:
def overlapscan(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Horizontal Off' in line:
                if verbose:
                    print(line)
                wide = int(line.split('=')[1])
                if wide == 1:
                    wide=False
    return(wide)

In [46]:
def threesixtyscan(logfile, verbose=False):
    threesixty = False
    with open(logfile, 'r') as f:
        for line in f:
            if '0 Rotation' in line:
                if verbose:
                    print(line)
                threesixty = line.split('=')[1].strip()
                if threesixty == 'YES':
                    threesixty = True
    return(threesixty)

In [47]:
Data['Stacks'] = [stacks(log) for log in Data['LogFile']]
Data['Wide'] = [overlapscan(log) for log in Data.LogFile]
Data['ThreeSixty'] = [threesixtyscan(log) for log in Data['LogFile']]

In [48]:
def exposure(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Exposure' in line:
                if verbose:
                    print(line)
                exp = int(line.split('=')[1])
    return(exp)

In [49]:
def averaging(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Avera' in line:
                if verbose:
                    print(line)
                details = line.split('=')[1]
                if 'ON' in details:
                    # https://stackoverflow.com/a/4894156/323100
                    avg = int(details[details.find("(")+1:details.find(")")])
                else:
                    avg=False
    return(avg)

In [50]:
Data['Exposure'] = [exposure(log) for log in Data['LogFile']]
Data['Averaging'] = [averaging(log) for log in Data['LogFile']]

In [51]:
def ringremoval(logfile, verbose=False):
    ring = numpy.nan
    with open(logfile, 'r') as f:
        for line in f:
            if 'Ring' in line:
                if verbose:
                    print(line)
                ring = int(line.split('=')[1].strip())
    return(ring)

In [52]:
def beamhardening(logfile, verbose=False):
    bh = numpy.nan
    with open(logfile, 'r') as f:
        for line in f:
            if 'ardeni' in line:
                if verbose:
                    print(line)
                bh = int(line.split('=')[1].strip())
    return(bh)

In [53]:
def get_reconstruction_grayvalue(logfile):
    grayvalue = None
    """How did we map the brightness of the reconstructions?"""
    with open(logfile, 'r') as f:
        for line in f:
            if 'Maximum for' in line:
                grayvalue = float(line.split('=')[1])
    return(grayvalue)

In [54]:
Data['RingRemoval'] = [ringremoval(log) for log in Data['LogFile']]
Data['Beamhardening'] = [beamhardening(log) for log in Data['LogFile']]
Data['GrayValueMax'] = [get_reconstruction_grayvalue(log) for log in Data['LogFile']]

In [55]:
import datetime
def duration(logfile, verbose=False):
    '''Returns scantime in *seconds*'''
    with open(logfile, 'r') as f:
        for line in f:
            if 'Scan duration' in line:
                if verbose:
                    print(line)
                duration = line.split('=')[1].strip()
    # Sometimes it's '00:24:26', sometimes '0h:52m:53s' :-/
    if 'h' in duration:
        scantime = datetime.datetime.strptime(duration, '%Hh:%Mm:%Ss')
    else:
        scantime = datetime.datetime.strptime(duration, '%H:%M:%S')
    return((scantime-datetime.datetime(1900,1,1)).total_seconds())

In [56]:
Data['Scan time'] = [get_scantime(log) for log in Data['LogFile']]
Data['Scan time total'] = [ st * stk  for st, stk in zip(Data['Duration'], Data['Stacks'])]

NameError: name 'get_scantime' is not defined

In [57]:
def nreconversion(logfile, verbose=False):
    Program = numpy.nan
    Version = numpy.nan
    with open(logfile, 'r') as f:
        for line in f:
            if 'Reconstruction Program' in line:
                if verbose:
                    print(line)
                Program = line.split('=')[1].strip()
            if 'Program Version' in line:
                if verbose:
                    print(line)
                Version = line.split('sion:')[1].strip()
    return(Program, Version)

In [58]:
Data['NRecon'] = [nreconversion(log) for log in Data['LogFile']]

In [59]:
Data.Voxelsize.unique()

array([5.00004 , 5.000024, 5.000018, 5.000336, 5.001531, 5.001393])

In [60]:
# Check voxel sizes (*rounded* to two after-comma values)
# If different, spit out which values
roundto = 2
if len(Data['Voxelsize'].round(roundto).unique()) > 1:
    print('We scanned all datasets with %s different voxel sizes' % len(Data['Voxelsize'].round(roundto).unique()))
    for vs in sorted(Data['Voxelsize'].round(roundto).unique()):
        print('-', vs, 'um for ', end='')
        for c, row in Data.iterrows():
            if float(vs) == round(row['Voxelsize'], roundto):
                print(os.path.join(row['Sample'], row['Scan']), end=', ')
        print('')
else:
    print('We scanned all datasets with equal voxel size, namely %s um.' % float(Data['Voxelsize'].round(roundto).unique()))

We scanned all datasets with equal voxel size, namely 5.0 um.


In [61]:
# Get an overview over the total scan time
# Nice output based on https://stackoverflow.com/a/8907407/323100
total_seconds = int(Data['Scan time total'].sum().total_seconds())
hours, remainder = divmod(total_seconds,60*60)
minutes, seconds = divmod(remainder,60)
print('In total, we scanned for %s hours and %s minutes)' % (hours, minutes))
for machine in Data['Scanner'].unique():
    total_seconds = int(Data[Data['Scanner'] == machine]['Scan time total'].sum().total_seconds())
    hours, remainder = divmod(total_seconds,60*60)
    minutes, seconds = divmod(remainder,60)
    print('\t - Of these, we scanned %s hours and %s minutes on the %s,'
          'for %s scans' % (hours,
                            minutes,
                            machine,
                            len(Data[Data['Scanner'] == machine])))

KeyError: 'Scan time total'

In [62]:
Data.to_excel('Details.xlsx')

In [63]:
Data.to_excel(os.path.join(Root,'Details.xlsx'))

Now we have all the data, but some of it is hidden in the `proj` and some of it in the `rec` log files...

----

My microct blurb from http://simp.ly/publish/NBhZhH

In [64]:
print('Based on the %s log files in %s' % (len(Data), Root))

Based on the 70 log files in /home/habi/P/Documents/Semela-Liver/Data/Liver-Semela


In [65]:
" OR ".join(str(value) for value in Data.Scanner.unique())

'SkyScan 1272'

In [67]:
print('After $PREPARATION, the',
      len(Data),
      'samples were imaged on a Bruker',
      " OR ".join(str(value) for value in Data.Scanner.unique()),
      'high-resolution microtomography machine (Control software version',
      " OR ".join(str(value) for value in Data.ControlSoftware.unique()) + 
      ', Bruker microCT, Kontich, Belgium).')

After $PREPARATION, the 70 samples were imaged on a Bruker SkyScan 1272 high-resolution microtomography machine (Control software version 1.1.9 OR 1.1.19, Bruker microCT, Kontich, Belgium).


In [68]:
print('The machine is equipped with a',
      " OR ".join(str(value) for value in Data.Source.unique()),
      'X-ray source and a',
      " OR ".join(str(value) for value in Data.Camera.unique()),
      'camera.')

The machine is equipped with a Hamamatsu L11871_20 X-ray source and a XIMEA xiRAY16 camera.


In [69]:
# if len(Data.Scanner.unique()) > 1:
#     print('more')

In [70]:
print('The X-ray source was set to a tube voltage of', 
      " OR ".join(str(value) for value in Data.Voltage.unique()),
      'kV and a tube current of',
      " OR ".join(str(value) for value in Data.Current.unique()),
      'µA, the x-ray spectrum was', end=' ')
if Data.Filter.unique():
    print('filtered by', " OR ".join(str(value) for value in Data.Filter.unique()), end=' ')
else:
    print('not filtered', end=' ')
print('prior to incidence onto the sample.')

The X-ray source was set to a tube voltage of 100.0 kV and a tube current of 100.0 µA, the x-ray spectrum was filtered by Cu 0.11mm prior to incidence onto the sample.


In [71]:
print('For each sample, we recorded a set of', end=' ')
if Data.Filter.unique().tolist():   
    print(" or ".join(str(value) for value in Data.Stacks.unique()),
          'stacked scans overlapping the sample height, each stack was recorded with', end=' ')
print(" or ".join(str(value) for value in Data.NumProj.unique()), 'projections of', end=' ')
for cs in Data.CamSize.unique():
    print(cs[0], end=' ')
print('x', end=' ')
for cs in Data.CamSize.unique():
    print(cs[1], end=' ')
print('pixels', end=' ')
if Data.Wide.unique().tolist():
    print('(' + " or ".join(str(value) for value in Data.Wide.unique()), 'projections stitched laterally)', end=' ')
print('at every',
       str(" or ".join(str(value) for value in Data.RotationStep.unique())) + '° over a ', end='')
if Data.ThreeSixty.unique().tolist():
     print('360°', end=' ')
else:
    print('180°', end=' ')
print('sample rotation.')

For each sample, we recorded a set of 2 or 3 stacked scans overlapping the sample height, each stack was recorded with 1895 or 962 or 974 projections of 4904 2452 4664 x 3280 1640 1638 pixels (False or 2 projections stitched laterally) at every 0.1 or 0.2° over a 360° sample rotation.


In [72]:
Data.ThreeSixty.unique().tolist()

['NO']

In [73]:
Data.Exposure.mean()

5946.042857142857

In [74]:
print('Every single projection was exposed for',
      " or ".join(str(value) for value in Data.Exposure.unique()),
      'ms,',
      " or ".join(str(value) for value in Data.Averaging.unique()),
      'projections were averaged to greatly reduce image noise.')

Every single projection was exposed for 5391 or 3923 or 4400 or 8866 ms, 3 projections were averaged to greatly reduce image noise.


In [75]:
log=Data['LogFile'][0]

In [76]:
stacks(log, verbose=True)

Number of connected scans=2



2

In [77]:
print('This resulted in a scan time of approximately ', end='')
if duration(log)/3600 > 1:
    # Scan took hours
    print(timeformat(datetime.timedelta(seconds=duration(log)),
                     '{hours} hours and {minutes} minutes'), end=' ')
else:
    print(timeformat(datetime.timedelta(seconds=duration(log)),
                     '{minutes} minutes'), end=' ')
if not stacks(log) == 1:
    print('per stack and about',
          timeformat(stacks(log) * datetime.timedelta(seconds=duration(log)),
                     '{hours} hours and {minutes} minutes'), end=' ')
print('per sample', end='')
if stacks(log) == 1:
    print('.')
else:
    print(' (with', stacks(log), 'stacks).')

This resulted in a scan time of approximately 

NameError: name 'timeformat' is not defined

In [78]:
len(Data.Scan.unique()), Data.Stacks.mean()

(3, 2.0714285714285716)

In [79]:
print('In total, we scanned', Data.Stacks.sum(), 'stacks.')
print('Each stack took approximately',
      Data.Duration.mean() // 60,
      'minutes (' + str(datetime.timedelta(seconds=Data.Duration.mean())) + ')')
print('In total, we thus scanned for about', 
      timeformat(Data.Stacks.sum() *
                 datetime.timedelta(seconds=Data.Duration.mean()),
                 '{days} days, {hours} hours and {minutes} minutes.'))
hourlyrate = 125
print('At the MIC rate of %s CHF/h, this would have cost %s CHF' % (
    hourlyrate,
    int(round(Data.Stacks.sum() * Data.Duration.mean() / 60 / 60 * hourlyrate))))

In total, we scanned 145 stacks.


AttributeError: 'DataFrame' object has no attribute 'Duration'

In [80]:
print('In total, we scanned %s samples at %s stacks on average' % (len(Data.Scan.unique()), Data.Stacks.mean()))
print('Each stack took approximately',
      Data.Duration.mean() // 60,
      'minutes (' + str(datetime.timedelta(seconds=Data.Duration.mean())) + ')')
print('In total, we thus scanned for about', 
      timeformat(len(Data.Scan.unique()) * Data.Stacks.mean() *
                 datetime.timedelta(seconds=Data.Duration.mean()),
                 '{days} days, {hours} hours and {minutes} minutes.'))
print('At the MIC rate, this would have cost',
      int(round(len(Data.Scan.unique()) * Data.Stacks.mean() * Data.Duration.mean() / 60 / 60 * 75)),
      'CHF.')

In total, we scanned 3 samples at 2.0714285714285716 stacks on average


AttributeError: 'DataFrame' object has no attribute 'Duration'

In [ ]:
Data.NRecon.unique()

In [ ]:
print('The projection images were then subsequently reconstructed into a 3D stack',
      'of images with',
      Data.Version.unique()[0][0],
      '(Version',
      version(log)[1] + ', Bruker microCT, Kontich Belgium)', end=' ')
if ringremoval(log):
    print('using a ring artifact correction of',
          ringremoval(log), end='')
if beamhardening(log):
    print(' and a beam hardening correction of',
          beamhardening(log),
          '%.')
else:
    print('.')
   

In [ ]:
print('The whole process resulted in datasets with an isometric voxel size of',
      " or ".join(str(value) for value in Data.Voxelsize_rounded.unique()),
      'µm.') 

In [ ]:
# fulllog(log)

In [ ]:
Data.Voxelsize.mean()

In [ ]:
Data[['Scan', 'Beamhardening', 'RingRemoval']]